In [53]:
# 

In [54]:

import googleapiclient.discovery
import googleapiclient.errors
from IPython.display import JSON
import pandas as pd 

In [ ]:
api_key = "Poner mi APi aqui "

#### ID Canal YouTube 

Para encontar el id del un canal es is al canal y la parte principal 
pulsas CTLR + U y buscar CTRL + F  : browse_id

- Video Completo : https://www.youtube.com/watch?v=Q2828ldk0Uc


In [56]:
channel_ids = ["UCaY_-ksFSQtTGk0y1HA_3YQ", 
                #Ibai
                "UCiyKCDvsHhXtGo0CuQIy8oQ", 
                #Yo
               ]

In [57]:
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name,api_version,developerKey=api_key
    )


In [58]:
def get_channel_stats(youtube, channel_ids):

    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
        )
    
    response = request.execute()


    #loop de los elementos
    for item in response ['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item["statistics"]['viewCount'],
            'totalViews':item['statistics']['videoCount'],
            'playlistId':item['contentDetails']['relatedPlaylists']['uploads']
        }

        all_data.append(data)

    return (pd.DataFrame(all_data))

    # JSON(response)

In [59]:
channel_stats = get_channel_stats(youtube, channel_ids)

In [60]:
channel_stats

,channelName,subscribers,views,totalViews,playlistId
0,Lino Ela,517,222369,92,UUiyKCDvsHhXtGo0CuQIy8oQ
1,Ibai,12200000,3834701998,1672,UUaY_-ksFSQtTGk0y1HA_3YQ


In [66]:
#playlist de videos 

playlist_id = "UUaY_-ksFSQtTGk0y1HA_3YQ"

def get_video_ids(youtube, playlist_id):

    video_ids = []

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId="UUaY_-ksFSQtTGk0y1HA_3YQ",
        maxResults = 15
    )
    
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get("nextPageToken")

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId="UUaY_-ksFSQtTGk0y1HA_3YQ",
            maxResults = 50,
            pageToken = next_page_token
            )
        
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get("nextPageToken")

    return video_ids



In [67]:
video_ids = get_video_ids(youtube,playlist_id)

KeyboardInterrupt: 

In [ ]:
len(video_ids)

['YiiYrdM7CGs', '64BaBMUYiDg', 'wxj-10sctYs', 'kXncKO2wedA', 'lEop6nQ-Paw']

### Obtner los Comentarios de los video

In [76]:
def get_comments_in_videos(youtube, video_ids):
    all_comments = []

    for video_id in video_ids:
        request = youtube.commentThreads().list(
            part = 'snippet, replies',
            videoId = video_id
        )

        response = request.execute()

        comments_in_video = [comment ['snippet']['topLevelComment']['snippet']['textOriginal']for comment in response['items'][0:10]]
        comments_in_video_info = {
            'video_id': video_id,
            'comments': comments_in_video
        }

        all_comments.append(comments_in_video_info)
    
    return pd.DataFrame(all_comments)



In [77]:
comments_df = get_comments_in_videos(youtube, video_ids)
comments_df

,video_id,comments
0,YiiYrdM7CGs,[Está mejorando su pronunciación y eso es muy ...
1,64BaBMUYiDg,"[Publicidad para gta 6? 🤓, Tremendo el gta6 😮,..."
2,wxj-10sctYs,[El truco es echarle a las galletas los sobres...
3,kXncKO2wedA,[Soñe que un dia ibai diria vamos a caminar 12...
4,lEop6nQ-Paw,"[""Nunca se sabe si hay una invasión marroquí ""..."


In [78]:
comments_df['comments'][0]

['Está mejorando su pronunciación y eso es muy bueno! Pero Ibai, in Spain and Latin America we speak spanish, not english 😅',
 'El Klk de ray😂😂',
 'Grande ibai',
 'Hello ibai',
 'Gigante noble en accion🔥👑',
 'Ojalá hubiera mandado a Kike otra vez',
 'Klk ya tu sabe',
 '“Para toda la gente de España y Latinoamérica” *procede a hablar en inglés*',
 'Que humilde Ibai',
 'Your videos are not just content, they are stories that stay in the heart. Keep inspiring us!♠️🤞🍪']

### Estadisticas de Videos

In [ ]:
#Estadisticas de Videos 

def get_video_details(youtube, video_ids):

    all_video_info = []

    for i in range(0, len(video_ids),50):
        request = youtube.videos().list(
            part='Snippet,contentDetails, statistics',
            id=','.join(video_ids[i:i+50])
        )

        response = request.execute()

        for video in response['items']:
            stats_to_keep = {
                'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                'statistics':['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                'contentDetails':['duration', 'definition', 'caption']
            }

            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except Exception: 
                        video_info[v] = None
                        

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

In [71]:
video_df = get_video_details(youtube, video_ids)

video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,YiiYrdM7CGs,Ibai,Hello guys,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-11-20T16:00:28Z,1532,576,None,21,PT29S,hd,false
1,64BaBMUYiDg,Ibai,Día 80 de mi cambio físico,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-11-19T16:00:03Z,287412,33366,None,298,PT59S,hd,false
2,wxj-10sctYs,Ibai,Probando el desayuno militar,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-11-18T15:01:07Z,96952,7893,None,52,PT53S,hd,false
3,kXncKO2wedA,Ibai,PASÉ 24H HACIENDO LA DIETA MILITAR,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-11-16T21:06:44Z,1600288,74995,None,1053,PT17M44S,hd,false
4,lEop6nQ-Paw,Ibai,Día 79 de mi cambio físico,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-11-15T16:00:48Z,1248068,93553,None,1016,PT59S,hd,false


### DATA PROCESSING 
